# 概念

1. 多个输入通道的情况：

每个通道都有一个卷积核，结果是所有的卷积结果的和（输出都是单通道）

2. 多个输出通道的情况：

对于之前的单通道输出，叠加对应的m个核生成m个输出通道


## 为什么要多通道

1. 每个输出通道可以识别特定模式



2. 所有输入通道核识别并组合输入中的模式


 和multihead selfattention一个意思


## 1*1 卷积层

不识别空间模式，只是做通道融合

# coding

In [2]:
import torch
from torch import nn

/opt/homebrew/Caskroom/miniforge/base/envs/Atari/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1.1 多个输入通道的处理情况

卷积结果直接加和

In [3]:
def corr2d(X, K):
    h, w = K.shape
    y = torch.zeros((X.shape[0] - h + 1), X.shape[1]-w+1)
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return y

def corr2d_multi_in(X, K):
    return sum(corr2d(x, k) for x, k in zip(X, K)) 

In [34]:
X = torch.tensor(range(9)).float().repeat(2,1)
X = X.reshape((2,3,3))
X[1] += 1

In [44]:
K = torch.tensor(range(4)).float().repeat(2,1).reshape((2,2,2))
K[1] += 1
K

tensor([[[0., 1.],
         [2., 3.]],

        [[1., 2.],
         [3., 4.]]])

In [38]:
Y = corr2d_multi_in(X, K)
Y

tensor([[ 56.,  72.],
        [104., 120.]])

In [39]:
X.shape, K.shape, Y.shape

(torch.Size([2, 3, 3]), torch.Size([2, 2, 2]), torch.Size([2, 2]))

## 1.2 多个输出通道

In [45]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K = torch.stack([K, K+1, K+2], 0)
K.shape

torch.Size([3, 2, 2, 2])

In [46]:
multi_Y = corr2d_multi_in_out(X, K)
multi_Y

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## 1.3 1 * 1卷积

In [49]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    
    Y = torch.matmul(K, X)
    
    return Y.reshape(c_o, h, w)

X = torch.normal(0, 1, (3,3,3))
K = torch.normal(0, 1, (2, 3, 1,1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

assert float(torch.abs(Y1-Y2).sum()) < 1e-6

如上，1 * 1的卷积，就等价于一个全连接层